# 🧹 Limpeza e Preparação dos Dados - VGChartz 2024

---

## Objetivo

Este notebook é responsável pela **limpeza, tratamento e preparação** do dataset bruto de vendas de videogames do VGChartz 2024. O objetivo é transformar os dados brutos em um dataset limpo e estruturado, pronto para análise estatística descritiva e modelagem de regressão linear.

## Estratégia de Limpeza

Devido à grande quantidade de dados faltantes no dataset original (64.016 jogos), optamos por uma abordagem focada:

**Focar nos Top 2000 jogos mais vendidos** para garantir:
- ✅ Qualidade dos dados (jogos com rastreamento completo)
- ✅ Relevância comercial (jogos que realmente impactaram o mercado)
- ✅ Dados confiáveis para regressão

## Critérios de Qualidade

Para que um registro seja mantido no dataset final, ele deve atender:

- ✅ Ter `critic_score` válido (essencial para regressão)
- ✅ Ter `total_sales` válido
- ✅ Estar entre os Top 2000 jogos mais vendidos
- ✅ Ter dados categóricos completos (`genre`, `console`, `publisher`)

---

## 1️⃣ Importação de Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

# Configurações de exibição
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

## 2️⃣ Carregamento do Dataset Original

In [2]:
load_dotenv()
data_raw_path = os.getenv("DATA_RAW_PATH")
df = pd.read_csv(data_raw_path)

df

,img,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
0,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.40,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
1,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.70,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
2,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.60,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN
3,/games/boxart/full_9218923AmericaFrontccc.jpg,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,NaN,15.86,9.06,0.06,5.33,1.42,2013-09-17,NaN
4,/games/boxart/full_4990510AmericaFrontccc.jpg,Call of Duty: Black Ops 3,PS4,Shooter,Activision,Treyarch,8.10,15.09,6.18,0.41,6.05,2.44,2015-11-06,2018-01-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64011,/games/boxart/full_2779838AmericaFrontccc.jpg,XBlaze Lost: Memories,PC,Visual Novel,Aksys Games,Arc System Works,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-11,2019-01-28
64012,/games/boxart/full_8031506AmericaFrontccc.jpg,"Yoru, Tomosu",PS4,Visual Novel,Nippon Ichi Software,Nippon Ichi Software,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-30,2020-05-09
64013,/games/boxart/full_6553045AmericaFrontccc.jpg,"Yoru, Tomosu",NS,Visual Novel,Nippon Ichi Software,Nippon Ichi Software,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-30,2020-05-09
64014,/games/boxart/full_6012940JapanFrontccc.png,Yunohana SpRING! ~Mellow Times~,NS,Visual Novel,Idea Factory,Otomate,NaN,NaN,NaN,NaN,NaN,NaN,2019-02-28,2019-02-24


## 3️⃣ Remoção de Colunas Desnecessárias

Removemos colunas que não serão utilizadas na análise estatística:

- `img`: URL da imagem do jogo (não relevante para análise)
- `last_update`: Data de atualização do banco de dados (metadado)

In [3]:
# Remover colunas desnecessárias
colunas_remover = ['img', 'last_update']
df = df.drop(columns=colunas_remover)


## 4️⃣ Extração do Ano de Lançamento

Criamos a variável `year` (quantitativa discreta) a partir de `release_date` para análise temporal.

In [4]:
# Extrair ano da data de lançamento
df['year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year

print("✓ Coluna 'year' criada com sucesso")
print(f"  Período: {int(df['year'].min())} - {int(df['year'].max())}")

✓ Coluna 'year' criada com sucesso
  Período: 1971 - 2024


## 5️⃣ Filtragem: Dados Essenciais para Regressão

### Passo 5.1: Remover jogos sem `critic_score` ou `total_sales`

Estas duas variáveis são **essenciais** para a análise de regressão linear (critic_score → total_sales).

In [5]:
# Filtrar apenas jogos com critic_score E total_sales válidos
df_clean = df.dropna(subset=['critic_score', 'total_sales']).copy()

print(f"\nJogos com critic_score E total_sales válidos: {len(df_clean):,}")
print(f"Redução: {len(df) - len(df_clean):,} jogos removidos")


Jogos com critic_score E total_sales válidos: 4,126
Redução: 59,890 jogos removidos


### Passo 5.2: Selecionar Top 2000 Jogos Mais Vendidos

**Justificativa Acadêmica:**
> "Focamos nossa análise nos 2000 jogos comercialmente mais relevantes (top vendas), pois possuem dados mais completos e confiáveis, garantindo maior robustez nas análises estatísticas"

In [6]:
# Selecionar Top 2000 jogos por vendas
TOP_N = 2000
df_clean = df_clean.nlargest(TOP_N, 'total_sales')

## 6️⃣ Análise de Valores Nulos

Verificamos quais colunas ainda possuem valores ausentes após a filtragem.

In [7]:
# Contagem de nulos
nulos = df_clean.isnull().sum()
total_registros = len(df_clean)
percent_nulos = (nulos / total_registros) * 100

# DataFrame resumo
resumo_nulos = pd.DataFrame({
    'Coluna': df_clean.columns,
    'Nulos': nulos.values,
    'Percentual': percent_nulos.values.round(2)
}).sort_values('Nulos', ascending=False)

print(resumo_nulos.to_string(index=False))

      Coluna  Nulos  Percentual
    jp_sales   1023       51.15
   pal_sales     55        2.75
    na_sales     32        1.60
 other_sales     26        1.30
       title      0        0.00
   developer      0        0.00
   publisher      0        0.00
       genre      0        0.00
     console      0        0.00
critic_score      0        0.00
 total_sales      0        0.00
release_date      0        0.00
        year      0        0.00


## 7️⃣ Tratamento de Vendas Regionais

### Problema Identificado

A coluna `jp_sales` (vendas no Japão) apresenta **51% de valores nulos**, tornando-a não confiável para análise.

### Decisão Tomada

1. **Remover `jp_sales`** (51% nulos - dados não confiáveis)
2. **Preencher com 0** os nulos restantes em outras regiões (< 3% cada)

In [8]:
# Remover jp_sales
df_clean = df_clean.drop(columns=['jp_sales'])


vendas_regionais = ['na_sales', 'pal_sales', 'other_sales']

# Preencher nulos restantes com 0
df_clean[vendas_regionais] = df_clean[vendas_regionais].fillna(0)
print("✓ Nulos restantes preenchidos com 0")

# Verificação final
print("\nNulos por região (DEPOIS):")
for col in vendas_regionais:
    nulos = df_clean[col].isnull().sum()
    print(f"  ✓ {col}: {nulos}")

✓ Nulos restantes preenchidos com 0

Nulos por região (DEPOIS):
  ✓ na_sales: 0
  ✓ pal_sales: 0
  ✓ other_sales: 0


## 8️⃣ Salvamento do Dataset Limpo

O dataset limpo será salvo em formato CSV para uso nos próximos notebooks de análise.

In [11]:
# Definir caminho de saída
output_path = os.getenv("DATA_PROCESSED_PATH", "../data/processed/vgchartz_clean.csv")

# Criar diretório se não existir
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Salvar dataset limpo
df_clean.to_csv(output_path, index=False)

print("=" * 80)
print("✅ DATASET SALVO COM SUCESSO")
print("=" * 80)

✅ DATASET SALVO COM SUCESSO


---

## 📝 Resumo do Processo de Limpeza

### Transformações Realizadas

1. ✅ **Remoção de colunas desnecessárias** (`img`, `last_update`)
2. ✅ **Extração da variável `year`** a partir de `release_date`
3. ✅ **Filtragem por dados essenciais** (critic_score + total_sales válidos)
4. ✅ **Seleção Top 2000** jogos mais vendidos
5. ✅ **Remoção de `jp_sales`** (51% nulos)
6. ✅ **Preenchimento de nulos** em outras regiões (< 3% cada)

### Dataset Final

- **Registros:** 2.000 jogos
- **Período:** 1993-2018
- **Qualidade:** 0 valores nulos
- **Foco:** Jogos comercialmente relevantes e bem rastreados

### Variáveis Disponíveis

**Quantitativas Contínuas (5):**
- `critic_score`, `total_sales`, `na_sales`, `pal_sales`, `other_sales`

**Quantitativa Discreta (1):**
- `year`

**Qualitativas (4):**
- `console`, `genre`, `publisher`, `developer`

---

